In [1]:
%matplotlib widget

import numpy as np
from pymcaspec import specfile
from pymcaspec.utils import calculate_dEdChan, bin_mythen, clean_mythen_data, construct_E_M
import matplotlib.pyplot as plt
import matplotlib as mpl

import ipympl

# All parameters here 
Unless otherwise specified Energy in keV distance in mm

In [10]:
# Analyzer related
a = 5.4309 # lattice constant Si in Ang
d = a/np.sqrt(8**2+4**2+4**2) # d spacing in Ang

analyzer_info = dict(d=d,  # d spacing in Ang
                    R=2000, # analyzer radius in mm
                    edgeEn=11.215 # edge energy in keV
                    )

magicchannel = 1050
# mythen related
mythen_clean_info = dict(
    min_chan = magicchannel - 50,
    max_chan = magicchannel + 50,
    threshold = 1000 # throw away values above this
)

monitor = 'i2' # name of monitor to use

# Define files to read in

In [11]:
scan_no = 103
filestem = '11Jul2018'
F_main = specfile(filestem)
filename = '{}.scan{}.MYT'.format(filestem, scan_no)
F = specfile(filename)

F_processed = specfile('{}.spm3'.format(filestem))

# Use Analyzer properties for Si(8 4 4)
to compute energy per channel

In [12]:
dEdchan = calculate_dEdChan(**analyzer_info)

# Read data

In [21]:
mythen_dataset = F.get_all_MCA()
central_Es = F_main[scan_no][0]
central_Ms = F_main[scan_no][monitor]
S_processed = F_processed[scan_no]

# Clean mythen data

In [7]:
mythen_dataset = clean_mythen_data(mythen_dataset, **mythen_clean_info)

# Contruct vectors

In [14]:
E_dataset, M_dataset = construct_E_M(central_Es, central_Ms, mythen_dataset, magicchannel, dEdchan)

# Bin data

In [22]:
central_Es.shape

(401,)

In [23]:
E_dataset.shape

(401, 1280)

In [28]:
mythen_dataset.shape

(402, 1280)

In [35]:
fig, ax = plt.subplots()
ax.plot(mythen_dataset[-1])
ax.plot(mythen_dataset[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
E, I, M = bin_mythen(E_dataset, M_dataset, mythen_dataset, binstep=dEdchan/2)

ValueError: weights should have the same shape as a.

In [ ]:
fig, ax = plt.subplots()

ax.plot(E, I, 'o', label='our_binning')

ax.plot(S_processed[0], S_processed['C'], 'x', label='processed')

ax.legend()